In [ ]:
import pandas as pd

file1_path = 'datasets/cleaned_100k.csv'
data1 = pd.read_csv(file1_path, dtype={'WineID': str}, low_memory=False)

file2_path = '/datasets/cleaned_ratings_file.csv'
data2 = pd.read_csv(file2_path, dtype={'WineID': str}, low_memory=False)

print("Columns in data1:", data1.columns)
print("Columns in data2:", data2.columns)

mean_ratings = data2.groupby('WineID')['Rating'].mean().reset_index()

result = pd.merge(data1, mean_ratings, on='WineID', how='left')

result.rename(columns={'Rating': 'Ratings'}, inplace=True)

print(result.head())

result.to_csv('updated_wines.csv', index=False)



# Pre Data Preprocessing - Task [92]

## Purpose

1. Merging both updated_wines.csv that has the mean ratings, with the merged_wine_dataset that was the result of Report 2(id:65). Adding the rating of the first dataset to the second.

In [ ]:
import pandas as pd

file1 = "datasets/updated_wines.csv"
file2 = "datasets/merged_wine_dataset.csv"

df1 = pd.read_csv(file1) 
df2 = pd.read_csv(file2) 

# Merge the datasets based on WineName and WineryName
merged_df = df2.merge(df1[['WineName', 'WineryName', 'Ratings']], on=['WineName', 'WineryName'], how='left')

# Save the new dataset
output_file = "datasets/PLNTD_dataset.csv"
merged_df.to_csv(output_file, index=False)

print(f"PLNTD_dataset created and saved to {output_file}")

missing_ratings = merged_df[merged_df['Ratings'].isna()]

#Testing purposes
if not missing_ratings.empty:
    print("WARNING: Some rows in the dataset are missing a rating.")
    print(missing_ratings)
else:
    print("SUCCESS: All rows have a rating.")


# Pre Data Preprocessing - Task [002]

## Purpose

1. Splitting text components for better granularity.
2. Normalizing numerical values to ensure consistency.
3. Removing unnecessary symbols or irrelevant text.

The preprocessing will address these specific attributes in the dataset:
- **Style**
- **Characteristics**
- **Price**
- **Capacity**
- **ABV (Alcohol by Volume)**
- **Vintage**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import datetime
import re

# Load the dataset
file_path = 'datasets/WineDataset.csv'
df = pd.read_csv(file_path)

def convert_to_liters(capacity):
    capacity = str(capacity).strip().upper()
    if 'CL' in capacity:  # Centiliters to Liters
        return float(re.sub(r'[^\d.]', '', capacity)) / 100
    elif 'ML' in capacity:  # Milliliters to Liters
        return float(re.sub(r'[^\d.]', '', capacity)) / 1000
    elif 'LITRE' in capacity or 'L' in capacity:  # Liters already
        return float(re.sub(r'[^\d.]', '', capacity))
    elif 'LTR' in capacity or 'L' in capacity:  # Liters already
        return float(re.sub(r'[^\d.]', '', capacity))
    elif 'L' in capacity or 'L' in capacity:  # Liters already
        return float(re.sub(r'[^\d.]', '', capacity))
    else:
        return ''  # Handle any unknown format

def preprocess_data(df):

    numeric_cols = ['Price', 'ABV', 'Capacity']

    df['Capacity'] = df['Capacity'].apply(convert_to_liters)

    if not df.empty:
        for col in numeric_cols:
            if col in df.columns:
                # Remove non-numeric characters and convert to float
                df[col] = df[col].apply(lambda x: re.sub(r'[^\d.]', '', str(x)).strip() if str(x).strip() else np.nan)
                df[col] = pd.to_numeric(df[col], errors='coerce')
                
                if df[col].notnull().any():  # Check if there's valid data for scaling
                    scaler = MinMaxScaler()
                    df[col] = scaler.fit_transform(df[[col]])
                
                df[col] = df[col].round(3)

        # Clean and split the 'Style' column
        if 'Style' in df.columns:
            df['Style'] = (
                df['Style']
                .str.replace(r'[^\w\s&]', '', regex=True)
                .str.split('&')
                .apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else x)  # Clean whitespace
            )

            # This code divides the 'Style' array into several columns, each representing a position in that array
            max_len = df['Style'].apply(lambda x: len(x) if isinstance(x, list) else 0).max()

            for i in range(1, max_len + 1):
                df[f'Style {i}'] = df['Style'].apply(lambda x: x[i-1] if isinstance(x, list) and len(x) >= i else '')

            df = df.drop(columns=['Style'])

        # Clean and split the 'Characteristics' column
        if 'Characteristics' in df.columns:
            df['Characteristics'] = (
                df['Characteristics']
                .str.replace(r'[^\w\s,]', '', regex=True)
                .str.split(',') 
                .apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else x)  # Clean whitespace
            )
            
            # This code divides the 'Characteristics' array into several columns, each representing a position in that array
            max_len = df['Characteristics'].apply(lambda x: len(x) if isinstance(x, list) else 0).max()

            for i in range(1, max_len + 1):
                df[f'Characteristic {i}'] = df['Characteristics'].apply(lambda x: x[i-1] if isinstance(x, list) and len(x) >= i else '')

            df = df.drop(columns=['Characteristics'])
            
        # Clean and normalize the 'Vintage' column
        if 'Vintage' in df.columns:
            current_year = datetime.datetime.now().year

            df['Vintage'] = df['Vintage'].apply(
                lambda x: current_year if str(x).strip().upper() == 'NV' else (int(re.search(r'\d{4}', str(x)).group(0)) if re.search(r'\d{4}', str(x)) else np.nan)
            )

            valid_years = df['Vintage'][df['Vintage'] > 1900]
            if not valid_years.empty:

                min_year = valid_years.min()  
                max_year = current_year

                # Calculates the vintage value based on the max vintage and the current year
                df['Vintage'] = df['Vintage'].apply(
                    lambda x: max(0, (x - max_year) / (min_year - max_year)) if pd.notna(x) else np.nan
                )

                # Round the 'Vintage' values to 2 decimal places
                df['Vintage'] = df['Vintage'].round(2)

    return df

# Preprocess the dataset
df_cleaned = preprocess_data(df)

# Save or display the cleaned dataset
df_cleaned.to_csv('datasets/cleaned_wines.csv', index=False)
df_cleaned.head()


In [ ]:
import pandas as pd

# Load your dataset
file_path = 'datasets/cleaned_wines.csv'
df = pd.read_csv(file_path)

# Get unique values in the 'Capacity' column
unique_values = df['Capacity'].dropna().unique()

# Print the unique values
print(unique_values)


In [ ]:
import pandas as pd

file2_path = '/datasets/XWines_Full_100k_wines.csv'

data2 = pd.read_csv(file2_path)

data2['WineID'] = pd.to_numeric(data2['WineID'], errors='coerce')

cleaned_data2 = data2.dropna(subset=['WineID'])

cleaned_file_path = 'cleaned_100k.csv'
cleaned_data2.to_csv(cleaned_file_path, index=False)

print(cleaned_data2.head())



In [ ]:
import pandas as pd

file2_path = '/datasets/updated_wines.csv'
data2 = pd.read_csv(file2_path, dtype={'WineID': str}, low_memory=False)

invalid_rows = data2[pd.to_numeric(data2['Ratings'], errors='coerce').isna()]

print("Rows with invalid or empty 'Ratings':")
print(invalid_rows)


In [ ]:
import pandas as pd

file2_path = 'datasets/updated_wines.csv'

data2 = pd.read_csv(file2_path, dtype={'WineID': str}, low_memory=False)

data2['Ratings'] = pd.to_numeric(data2['Ratings'], errors='coerce')

data2['Ratings'] = data2['Ratings'].apply(lambda x: f"{x:.2f}" if pd.notna(x) else "")

data2.to_csv(file2_path, index=False)

print(f"Ratings formatted to two decimal places. The original file has been updated: {file2_path}")
